<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled162.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit qiskit-aer numpy pandas matplotlib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 132.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import datetime
import random
import pickle
from qiskit import QuantumCircuit
from qiskit.providers.aer import AerSimulator

# --- 1. Quantum Random Number Generator ---
class QuantumRandomGenerator:
    def __init__(self, num_bits):
        self.num_bits = num_bits
        self.simulator = AerSimulator()

    def get_seed(self):
        # Create a quantum circuit
        circuit = QuantumCircuit(self.num_bits, self.num_bits)
        circuit.h(range(self.num_bits))
        circuit.measure(range(self.num_bits), range(self.num_bits))

        # Execute the circuit
        job = self.simulator.run(circuit, shots=1, memory=True)
        result = job.result()

        # Check if memory is not empty before accessing
        memory = result.get_memory()
        if not memory:
            print("Error: result.get_memory() returned an empty list.")
            print(f"Result object: {result}")
            return None # Or handle this case appropriately, maybe raise an error

        # Convert the binary string result to an integer
        binary_string = memory[0]
        return int(binary_string, 2)


# --- 2. Environment ---
class Environment:
    def __init__(self, rng, days_per_cycle=10):
        self.days_per_cycle = days_per_cycle
        self.time = 0
        self.rng = rng
        self.uv_intensity, self.temperature, _ = self._generate_cycles(
            rng, days_per_cycle, 365 * days_per_cycle, 50
        )

    def _generate_cycles(self, rng, days_per_cycle, steps_per_year, num_years):
        daily_uv_cycle = np.sin(np.linspace(0, 2 * np.pi, days_per_cycle))
        daily_uv_cycle[daily_uv_cycle < 0] = 0
        daily_temp_swing = -4 * np.cos(np.linspace(0, 2 * np.pi, days_per_cycle))

        full_uv, full_temp, yearly_severity_log = [], [], []

        for _ in range(num_years):
            uv_severity = rng.uniform(low=0.6, high=1.4)
            yearly_severity_log.extend([uv_severity] * steps_per_year)

            seasonal_temp_base = 15 - 10 * np.cos(np.linspace(0, 2 * np.pi, steps_per_year))
            seasonal_amplitude_mod = (0.225 * np.sin(np.linspace(0, 2 * np.pi, steps_per_year)) + 0.725) * uv_severity

            GEOTHERMAL_BASE_TEMP = 4.0

            for i in range(steps_per_year):
                cloud_cover_factor = rng.uniform(low=0.7, high=1.0)
                solar_temp = seasonal_temp_base[i] + daily_temp_swing[i % days_per_cycle]
                full_temp.append(max(GEOTHERMAL_BASE_TEMP, solar_temp))

                daily_uv = daily_uv_cycle[i % days_per_cycle]
                seasonal_uv = daily_uv * seasonal_amplitude_mod[i]
                final_uv = seasonal_uv * cloud_cover_factor
                full_uv.append(final_uv)

        return np.array(full_uv), np.array(full_temp), np.array(yearly_severity_log)

    def get_current_uv(self):
        return self.uv_intensity[(self.time + 1) % len(self.uv_intensity)]

    def get_current_temperature(self):
        return self.temperature[(self.time + 1) % len(self.temperature)]

    def step(self):
        self.time += 1


# --- 3. Protoribosome ---
class Protoribosome:
    def __init__(self, env, strategy, initial_sequence, rng, initial_mass=100.0):
        self.env = env
        self.strategy = strategy
        self.rna_sequence = list(initial_sequence)
        self.rng = rng
        self.rna_mass = initial_mass
        self.atp_pool = 500.0
        self.rna_damage_level = 0.0
        self.location = "shadow_zone"
        self.status = "ACTIVE"
        self.replication_rate = 0.05
        self.damage_tolerance_threshold = 3.0

    def step(self):
        if self.status == "INACTIVE":
            return None

        current_uv = self.env.get_current_uv()
        self.atp_pool = self.atp_pool * 0.975 + 1.5

        if self.location == "sunlight_zone" and current_uv > 0.1:
            self.rna_damage_level += current_uv * 0.1
        else:
            self.rna_damage_level = max(0, self.rna_damage_level - 0.05)

        if self.atp_pool < 10 or self.rna_damage_level > self.damage_tolerance_threshold:
            self.status = "INACTIVE"
            return None

        if self.atp_pool > 400 and self.rna_mass > 80 and self.rng.random() < self.replication_rate:
            offspring_mass = self.rna_mass * 0.5
            self.rna_mass -= offspring_mass
            self.atp_pool -= 100
            return Protoribosome(self.env, self.strategy, "".join(self.rna_sequence), self.rng, initial_mass=offspring_mass)

        return None

# --- 4. Colony ---
class Colony:
    def __init__(self, env, dna_template, initial_pop_size, steps_per_day, rng, max_population=1000):
        self.env = env
        self.dna_template = dna_template
        self.active_population = [Protoribosome(env, "cautious" if i % 2 == 0 else "readthrough", dna_template, rng) for i in range(initial_pop_size)]
        self.inactive_population = []
        self.rng = rng
        self.max_population = max_population
        self.steps_per_day = steps_per_day

    def step(self, current_step):
        if current_step > 0 and current_step % self.steps_per_day == 0:
            self.inactive_population.clear()

        next_generation = []
        offspring_cache = []

        for p in self.active_population:
            new_offspring = p.step()
            if p.status != "INACTIVE":
                next_generation.append(p)
            else:
                self.inactive_population.append(p)
            if new_offspring:
                offspring_cache.append(new_offspring)

        next_generation.extend(offspring_cache)

        if len(next_generation) > self.max_population:
            self.active_population = self.rng.sample(next_generation, self.max_population)
        else:
            self.active_population = next_generation

    def get_aggregated_data(self):
        cautious = sum(1 for p in self.active_population if p.strategy == "cautious")
        readthrough = len(self.active_population) - cautious
        return {'cautious': cautious, 'readthrough': readthrough}


# --- 5. Simulation Runner ---
def run_and_analyze_replica(replica_id, qrg, save_path):
    STEPS_PER_DAY = 10

    print(f"--- [Replica #{replica_id}] Generating new quantum seed... ---")
    quantum_seed = qrg.get_seed()
    print(f"Seed for Replica #{replica_id}: {quantum_seed}")

    rng = random.Random(quantum_seed)

    print(f"--- [Replica #{replica_id}] Starting New Simulation Run ---")
    data_log = []

    IDEAL_DNA_TEMPLATE = "AUGUGUUACUGG"
    env = Environment(rng)
    # Using the smaller population sizes for stability
    colony = Colony(env, IDEAL_DNA_TEMPLATE, initial_pop_size=500, steps_per_day=STEPS_PER_DAY, rng=rng, max_population=1000)

    step_count = 0
    winner = "Unknown"

    pbar = tqdm(desc=f"Replica #{replica_id} Steps")
    while True:
        agg_data = colony.get_aggregated_data()

        if step_count > 1000:
            if agg_data['cautious'] == 0 and agg_data['readthrough'] == 0:
                winner = "Mutual Extinction"; break
            elif agg_data['cautious'] == 0:
                winner = "Readthrough"; break
            elif agg_data['readthrough'] == 0:
                winner = "Cautious"; break

        colony.step(step_count)
        env.step()

        log_entry = {'time': env.time, **agg_data}
        data_log.append(log_entry)

        pbar.set_postfix(agg_data)
        pbar.update(1)
        step_count += 1

    pbar.close()
    final_day = (step_count - 1) // STEPS_PER_DAY
    print(f"--- [Replica #{replica_id}] KNOCKOUT on day {final_day}: {winner} wins. ---")

    details_df = pd.DataFrame(data_log)
    state_to_save = {"step": step_count, "log": details_df, "status": "completed", "winner": winner, "duration_days": final_day}

    with open(save_path, 'wb') as f:
        pickle.dump(state_to_save, f)

    summary = {"Replica ID": replica_id, "Winner": winner, "Duration (Days)": final_day}
    return summary, details_df


# --- 6. Main Orchestrator ---
def main():
    print("--- Python Simulation Script ---")

    NUM_REPLICAS = 10
    BASE_SAVE_DIR = "Python_Quantum_Experiments"
    os.makedirs(BASE_SAVE_DIR, exist_ok=True)

    parent_run_dir = os.path.join(BASE_SAVE_DIR, "Experiment_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
    os.makedirs(parent_run_dir, exist_ok=True)
    print(f"--- Starting new experiment. Files will be saved in: {parent_run_dir} ---")

    qrg = QuantumRandomGenerator(32)
    all_summaries = []

    for i in range(1, NUM_REPLICAS + 1):
        replica_id = f"{i:02d}"
        replica_dir = os.path.join(parent_run_dir, f"Replica_{replica_id}")
        os.makedirs(replica_dir, exist_ok=True)
        save_file_path = os.path.join(replica_dir, "simulation_state.pkl")

        summary, details_df = run_and_analyze_replica(replica_id, qrg, save_file_path)
        all_summaries.append(summary)

        if not details_df.empty:
            plt.figure()
            plt.plot(details_df['time'], details_df['cautious'], label="Cautious")
            plt.plot(details_df['time'], details_df['readthrough'], label="Readthrough")
            plt.title(f"Replica #{replica_id} Population")
            plt.xlabel("Time (Steps)")
            plt.ylabel("Count")
            plt.legend()
            plot_path = os.path.join(replica_dir, "plot.png")
            plt.savefig(plot_path)
            plt.close()
            print(f"--- Plot saved for Replica #{replica_id} ---")

    print("\n--- FINAL EXPERIMENT SUMMARY ---")
    summary_df = pd.DataFrame(all_summaries)
    print(summary_df)

# --- Run the main function ---
if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'qiskit.providers.aer'